## Imports
- Sets up Keras Core backend to use Torch.

In [ ]:
import numpy as np
import os
# need to define backend before importing Keras
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras_core as keras
from keras_core import layers
from keras_core import ops
import shutil
from PIL import Image
import keras_cv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Load Dataset

The data is organized under the `./../face-direction-dataset` folder with one subfolder per class (`up`, `straight`, `left` and `right`).
```
face-direction-dataset
├── up
│   ├── phoebe_up_happy_open_2.pgm
│   ...
├── straight
│   ├── mitchell_straight_happy_open_4.pgm
│   ├── phoebe_straight_neutral_open_4.pgm
│   ...
├── left
│   ├── phoebe_left_angry_sunglasses_2.pgm
│   ...
├── right
│   ├── phoebe_right_sad_sunglasses.pgm
│   ...
```

In [ ]:
train_dataset, validation_dataset = keras.utils.image_dataset_from_directory(
    directory='./../face-direction-dataset/',
    labels="inferred",
    label_mode='categorical',
    # NOTE: you can adjust the batch_size to tune the model training process
    batch_size=32,
    # NOTE: the image size here is fixed because this is the size of the ImageNet images
    image_size=(224, 224),
    validation_split=0.2,
    subset="both",
    seed=0
)

## Visualize Dataset
Viewing the images within the dataset

In [ ]:
# Plots images in a grid
def plot_image_grid(images, grid=3, title=None):
    fig, axes = plt.subplots(grid,grid, figsize=(grid*2,grid*2))
    for i in range(grid):
        for j in range(grid):
            if i*grid + j < len(images):
                axes[i][j].imshow(images[i*grid + j].astype('uint8'))
    if title is not None:
        fig.suptitle(title)
    plt.tight_layout()
    
X = np.concatenate([x for x, y in train_dataset], axis=0)
plot_image_grid(X, title="directions of faces")

## Load Classification Model
Load a pre-trained model provided by Keras. Feel free to use any model, we tested this example with the `keras.applications.ResNet50()` model, and you can see how to use it here: https://keras.io/api/applications/resnet/#resnet50-function. 

In [ ]:
# TODO: load a pre-trained classification model
# TODO: remove the last layer of the model so we change change the number of classes
resnet = 
# NOTE: with ResNet50(), there is an option to remove the last layer when loading the pre-trained model

## Build Transfer Learning Model
Freeze the layers in the pre-trained model by setting the model's or layer's `.trainable` to be `False`, and add a few trainable layers to the end of the model for our training process using the Sequential or Functional API. You can try adding one `Conv2D()` layer, one `Flatten()` layer, and one `Dense()` layer. Remember that the last `Dense()` layer needs to be updated to predict 4 classes.

In [ ]:
# TODO: set the model.trainable to be False

In [ ]:
# TODO: build the transfer learning model by adding layers to the end of the `resnet` model instantiated above

# NOTE: you can use the Sequential or Functional API to define the layers here

## Train Classification Model
Define the optimizer and loss to use for the training process with `model.compile()`, and run the training loop on the dataset with `model.fit()`

In [ ]:
# TODO: Compile the model with model.compile and with the optimizer, loss, and 'accuracy' metrics

In [ ]:
# Should be able to get at least 70% for validation accuracy

# TODO: train the model with model.fit and the train/validation dataset
# TODO: you will also need to set the number of training epochs
history = 

# NOTE: 1-5 epochs should be suitable for this task based on the learning rate of your optimizer.
# NOTE: training could take a while since we are using a large pre-trained model.

In [ ]:
# Load the training history into a pandas.DataFrame
history_df = pd.DataFrame(history.history)
history_df.head(3)

In [ ]:
# Helper function to plot accuracy and loss of the run
def plot_history(history_df):
    fig, axes = plt.subplots(2,1, sharex=True)
    sns.lineplot(data=history_df[["accuracy", "val_accuracy"]], ax=axes[0]).set(
        title="accuracy over iterations of training",
        xlabel="iterations",
        ylabel="accuracy"
    )
    sns.lineplot(data=history_df[["loss", "val_loss"]], ax=axes[1]).set(
        title="loss over iterations of training",
        xlabel="iterations",
        ylabel="loss"
    )
    plt.tight_layout()
plot_history(history_df)

## Analyze the Model
Plotting a confusion matrix and visualizing incorrectly classified images

In [ ]:
labels = sorted(["left", "right", "up", "straight"])

In [ ]:
# Analyzing the model through a confusion matrix
def confusion_matrix(y_true, y_pred):
    labels = np.unique(y_true)
    matrix = np.zeros((len(labels), len(labels)))
    for i, label_true in enumerate(labels):
        for j, label_pred in enumerate(labels):
            matrix[i][j] = np.count_nonzero((y_true == label_true) & (y_pred == label_pred))
    return matrix

def get_confusion_matrix(model, dataset):
    y_true = np.argmax(np.concatenate([y for x, y in dataset], axis=0), axis=1)
    y_pred = np.argmax(model.predict(dataset), axis=1)
    confusion_mat = confusion_matrix(y_true, y_pred)
    confusion_df = pd.DataFrame(confusion_mat, columns=labels, index=labels)
    sns.heatmap(confusion_df, annot=True).set(
        title="Confusion Matrix",
        xlabel="Predictions",
        ylabel="Ground Truth"
    )
get_confusion_matrix(model, validation_dataset)

In [ ]:
y_true = np.argmax(np.concatenate([y for x, y in validation_dataset], axis=0), axis=1)
X = np.concatenate([x for x, y in validation_dataset], axis=0)
y_pred = np.argmax(model.predict(validation_dataset), axis=1)
predictions = list(zip(X, y_true, y_pred))

In [ ]:
predictions1 = [image for image, y_true, y_pred in predictions if y_pred == 2]
plot_image_grid(predictions1, grid=3, title="predicted looking straight")

In [ ]:
predictions1 = [image for image, y_true, y_pred in predictions if y_pred == 0]
plot_image_grid(predictions1, grid=3, title="predicted looking left")